In [1]:
import numpy as np
import pandas as pd
import time
import math
import itertools
import os

In [2]:
# Importing functions from the modules in the qseg package
# from qseg.graph_utils import image_to_grid_graph, draw, draw_graph_cut_edges
# from qseg.dwave_utils import dwave_solver, annealer_solver
# from qseg.utils import decode_binary_string

# Additional necessary imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from qiskit_optimization.applications import Maxcut
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [3]:
import gurobipy as gp
from gurobipy import GRB

import os
import time
import csv

import itertools

import random as rnd

In [4]:
from datetime import datetime

In [5]:
import utils
import min_cut_solvers
# import Utils_Solvers

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sgp4.api import Satrec, jday

# Fetch TLE data from CelesTrak
import requests

In [7]:
# def evaluateSplits(coalition, coalition_values, **kwargs):
#     agents = coalition.split(',')
#     n_agents = len(agents)
#     best_cost_brute = f[coalition]
#     xbest_brute = [coalition]
#     for b in range(1, 2**(n_agents-1)):
#         x = [int(term) for term in reversed(list(bin(b)[2:].zfill(n_agents)))]
#         first_half = ','.join([agent for i,agent in enumerate(agents) if int(x[i])])
#         second_half = ','.join([agent for i,agent in enumerate(agents) if not int(x[i])])
#         if best_cost_brute <= (f[first_half]+f[second_half]):
#             best_cost_brute = f[first_half]+f[second_half]
#             xbest_brute = [first_half, second_half]
#     return xbest_brute, best_cost_brute

In [8]:
# def IDP(coalition_values, evaluateSplits = evaluateSplits, min_cut_solver = min_cut_solvers.min_cut_brute_force, **kwargs):
#     n_agents = math.ceil(math.log(len(coalition_values),2))
#     global t
#     t = {}
#     global f
#     f = {}
#     for coalition,coalition_value in coalition_values.items():
#         t[coalition] = [coalition]
#         f[coalition] = coalition_value
#     for coalition_size in range(2, n_agents):
#         if((math.ceil((2*n_agents)/3)<coalition_size) and (coalition_size < n_agents)):                  # Ignoring this condition will make this function work as DP instead of IDP
#             continue
#         coalitions_of_cur_size = list(itertools.combinations(map(str,range(1,n_agents+1)), coalition_size))
#         for curCoalition in coalitions_of_cur_size:
#             curCoalition = ','.join(curCoalition)
#             split_t, split_f = evaluateSplits(curCoalition, coalition_values, min_cut_solver = min_cut_solver, **kwargs)
#             if split_f > f[curCoalition]:
#                 t[curCoalition] = split_t
#                 f[curCoalition] = split_f
#     grand_coalition = ','.join(map(str,range(1,n_agents+1)))

#     split_t, split_f = evaluateSplits(grand_coalition, coalition_values, min_cut_solver = min_cut_solver, **kwargs)
#     if split_f > f[grand_coalition]:
#         t[grand_coalition] = split_t
#         f[grand_coalition] = split_f
#     temp = t[grand_coalition].copy()
#     optimal_cs = []
#     while(len(temp)):
#         C = temp.pop()
#         if len(t[C])==1:
#             optimal_cs+=t[C]
#         if(len(t[C])!=1):
#             temp += t[C]
#     optimal_cs_value = sum([f[coalition] for coalition in optimal_cs])
#     return optimal_cs, optimal_cs_value

#### IDP Top-down approach (for ISGs only)

In [9]:
# def get_coalition_value(coalition, induced_subgraph_game):
#     agents = coalition.split(',')
#     return sum([induced_subgraph_game[','.join(map(str,sorted(map(int,key))))] for key in itertools.combinations(agents, 2)])

In [10]:
# def evaluateSplits_min_cut(coalition, induced_subgraph_game, min_cut_solver = min_cut_solvers.min_cut_brute_force, **kwargs):
#   #print("coalition",coalition,end='=')
#     global split_counter
#     agents = coalition.split(',')
#     n = len(agents)
#     if n==1:
#         return [coalition], 0
#     if n==2:
#         c_value = induced_subgraph_game[coalition]
#         if c_value<=0:
#             #print([agents[0],agents[1]], 0)
#             return [agents[0],agents[1]], 0
#         else:
#             #print([coalition], c_value)
#             return [coalition], c_value
#     min_cut_mapping = {}
#     for idx,agent in enumerate(agents):
#         min_cut_mapping[agent] = str(idx+1)
#     subproblem_as_induced_subgraph_game = {','.join([min_cut_mapping[vertex] for vertex in map(str,sorted(map(int,key)))]):induced_subgraph_game[','.join(map(str,sorted(map(int,key))))] for key in itertools.combinations(agents, 2)}
#     split_counter += 1
#     xbest_brute, best_cost_brute = min_cut_solver(n,subproblem_as_induced_subgraph_game, **kwargs)
#     if 0 in xbest_brute and 1 in xbest_brute:
#         first_half = ','.join([agent for idx,agent in enumerate(agents) if xbest_brute[idx]])
#         second_half = ','.join([agent for idx,agent in enumerate(agents) if not xbest_brute[idx]])
#         bruteforce_solution_decoded = [first_half, second_half]
#         best_cost_brute = get_coalition_value(first_half, induced_subgraph_game) + get_coalition_value(second_half, induced_subgraph_game)
#     else:
#         bruteforce_solution_decoded = [coalition]
#         best_cost_brute = get_coalition_value(coalition, induced_subgraph_game)
#     #print(bruteforce_solution_decoded, best_cost_brute)
#     return bruteforce_solution_decoded, best_cost_brute

In [11]:
# def IDP_min_cut_top_down(induced_subgraph_game, min_cut_solver = min_cut_solvers.min_cut_brute_force, **kwargs):
#     grand_coalition = ','.join(map(str,sorted(map(int,(set([key.split(',')[i] for i in range(2) for key in induced_subgraph_game]))))))
#     temp = [grand_coalition]
#     optimal_cs = []
#     while(len(temp)):
#         c = temp.pop()
#         c_split_t,c_split_f = evaluateSplits_min_cut(c, induced_subgraph_game, min_cut_solver = min_cut_solver, **kwargs)
#         if len(c_split_t)==1:
#             optimal_cs+=c_split_t
#         if len(c_split_t)>1:
#             temp += c_split_t
#     return optimal_cs, sum([get_coalition_value(c, induced_subgraph_game) for c in optimal_cs])

In [12]:
def dwave_solver(linear, quadratic, private_token, runs=1000, **kwargs):
  vartype = dimod.BINARY
  bqm = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, vartype)
  start_time = time.time()
  dwave_sampler = DWaveSampler(token = private_token, solver={'topology__type': 'pegasus'})
  connection_time = time.time() - start_time

  start_time = time.time()
  sampler = EmbeddingComposite(dwave_sampler)
  embedding_time = time.time() - start_time
  start_time = time.time()
  sample_set = sampler.sample(bqm, num_reads=runs)
  response_time = time.time() - start_time
  return sample_set, connection_time, embedding_time, response_time

def annealer_solver(G, private_token, n_samples=1000, **kwargs):
  start_time = time.time()
  w = -1 * nx.adjacency_matrix(G).todense()
#   w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  problem_formulation_time = time.time() - start_time
  sample_set, connection_time, embedding_time, response_time = dwave_solver(linear, quadratic, private_token, runs=n_samples)
  info_dict = sample_set.info['timing'].copy()

  start_time = time.time()
  samples_df = sample_set.to_pandas_dataframe()
  sample_fetch_time = time.time() - start_time

  info_dict['problem_formulation_time'] = problem_formulation_time
  info_dict['connection_time'] = connection_time
  info_dict['embedding_time'] = embedding_time
  info_dict['response_time'] = response_time
  info_dict['sample_fetch_time'] = sample_fetch_time
  return samples_df, info_dict



def get_optimal_bipartite(G):
    # Placeholder for the actual bipartition algorithm.
    # This function should return a binary string indicating the bipartition.
    nodes = list(G.nodes)
    mid = len(nodes) // 2
    bipartition = ['0'] * len(nodes)
    for i in range(mid):
        bipartition[i] = '1'
    return ''.join(bipartition)

def apply_bipartition(G, binary_string):
    part1, part2 = [], []
    nodes = list(G.nodes)
    for i, bit in enumerate(binary_string):
        if bit == '0':
            part1.append(nodes[i])
        else:
            part2.append(nodes[i])
    return part1, part2

def get_optimal_coalition_structure_annealer(G, n_max):
    global split_counter
    global base_path
    global qputimes_filename
    connected_components = list(nx.connected_components(G))
    partitions = []

    for component in connected_components:
        subgraph = G.subgraph(component).copy()
        if len(subgraph.nodes) <= n_max:
            partitions.append(list(subgraph.nodes))
        else:
            stack = [subgraph]
            while stack:
                current_subgraph = stack.pop()
                if len(current_subgraph.nodes) <= n_max:
                    partitions.append(list(current_subgraph.nodes))
                else:
                    # binary_string = get_optimal_bipartite(current_subgraph)
                    split_counter += 1
                    sample_set_df, info_dict = annealer_solver(current_subgraph, private_token='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', n_samples = 1000)
                    qputime_dic = info_dict.copy()
                    qputime_dic['split_index'] = split_counter
                    # sample_set_df, info_dict = annealer_solver(current_subgraph, private_token='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', n_samples = 1000)
                    # dwave_annealer_solution_first = ''.join(sample_set_df.iloc[0][:-3].astype(int).astype(str))
                    for i in range(len(sample_set_df)):
                        solution = ''.join(sample_set_df.iloc[i][:-3].astype(int).astype(str))
                        if len(set(solution)) == 2:
                            dwave_annealer_solution_first = solution
                            break
                    dwave_annealer_value_first = sample_set_df.iloc[0][-2:-1][0]
                    qputime_dic = info_dict.copy()
                    qputime_dic['n'] = len(current_subgraph.nodes)
                    sample_set_df.to_csv(os.path.join(base_path, f'sampleSet_{split_counter}.csv'), index=False)
                    with open(qputimes_filename, mode='a', newline='') as qpu_times_file:
                        writer = csv.DictWriter(qpu_times_file, fieldnames=qputime_dic.keys())
                        writer.writerow(qputime_dic)
                    part1, part2 = apply_bipartition(current_subgraph, dwave_annealer_solution_first)
                    # print("part1, part2",part1, part2)
                    subgraph1 = current_subgraph.subgraph(part1).copy()
                    subgraph2 = current_subgraph.subgraph(part2).copy()
                    stack.append(subgraph1)
                    stack.append(subgraph2)

    return partitions

In [39]:
def gurobi_qubo_solver(G):
  w = -1 * nx.adjacency_matrix(G).todense()
  # w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)

  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}

  qubo_matrix = np.zeros([len(linear),len(linear)])
  for key,value in linear.items():
    qubo_matrix[int(key),int(key)] = value
  for key,value in quadratic.items():
    qubo_matrix[int(key[0]),int(key[1])] = value/2
    qubo_matrix[int(key[1]),int(key[0])] = value/2

  n = qubo_matrix.shape[0]
  model = gp.Model()
  x = model.addVars(n, vtype=GRB.BINARY)
  obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
  model.setObjective(obj_expr)
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    solution = [int(x[i].X) for i in range(n)]
    binary_string = ''.join(str(bit) for bit in solution)
    return binary_string, model.objVal
  else:
    return None, None

def get_optimal_coalition_structure_gurobi(G, n_max):
    connected_components = list(nx.connected_components(G))
    partitions = []

    for component in connected_components:
        subgraph = G.subgraph(component).copy()
        if len(subgraph.nodes) <= n_max:
            partitions.append(list(subgraph.nodes))
        else:
            stack = [subgraph]
            while stack:
                current_subgraph = stack.pop()
                if len(current_subgraph.nodes) <= n_max:
                    partitions.append(list(current_subgraph.nodes))
                else:
                    binary_string, _ = gurobi_qubo_solver(current_subgraph)
                    part1, part2 = apply_bipartition(current_subgraph, binary_string)
                    # print("part1, part2",part1, part2, _)
                    subgraph1 = current_subgraph.subgraph(part1).copy()
                    subgraph2 = current_subgraph.subgraph(part2).copy()
                    stack.append(subgraph1)
                    stack.append(subgraph2)

    return partitions

In [14]:
def generate_induced_subgraph_game(distribution, n_agents, sparsity=0.0, seed = 123, **kwargs):
    """
    generate_induced_subgraph_game: To generate a . 
    :distribution: distribution function name to sample the edge weights from. (dtype: function) 
    :n_agents: number of agents (dtype: int) 
    :sparsity: describes the connectivity of the graph edges. \
    If 0.0, generates a fully connected graph. \
    If 1.0 generates a connected graph with minimal edges. 
    (dtype: float, range: [0,1]) 
    :returns: induced subgraph game with agents pairs as key and their synergy as float value. 
    (dtype: dictionary) 
    """
    induced_subgraph_game = {}
    #### D: tyring to set a seed to get compareable results
    connected_edges = [(i + 1, j + 1) for i, j in nx.random_tree(n_agents, seed=seed).edges] 
    extra_edges = list(set(itertools.combinations(range(1, n_agents + 1), 2)).difference(connected_edges))
    #### D: seed
    rnd.seed(seed)
    sampled_edges = rnd.sample(extra_edges, int((1 - sparsity) * len(extra_edges)))
    keys = sorted(connected_edges + sampled_edges)
    totalinteractions = len(keys)
    np.random.seed(seed=seed)
    values = distribution(totalinteractions, **kwargs)
    for i, key in enumerate(keys):
        induced_subgraph_game[','.join(map(str, key))] = round(values[i], 2)
    G=nx.Graph()
    G.add_nodes_from(np.arange(0,n_agents,1))
    elist = [tuple((int(x)-1 for x in key.split(',')))+tuple([induced_subgraph_game[key]*-1]) for key in induced_subgraph_game]
    G.add_weighted_edges_from(elist)
    return induced_subgraph_game, G

### Data generation

In [15]:
# def normal(size=1, mu=0, sigma=5, low=-10, high=10):
#     values = np.random.normal(mu, sigma, size)
#     #values = np.interp(values, (values.min(), values.max()), (low, high))
#     return values

# def uniform(size=1, low=-5, high=5):
#     values = np.random.normal(low, high, size)
#     #values = np.interp(values, (values.min(), values.max()), (low, high))
#     return values

# def laplace(size=1, loc=0, scale=5, low=-10, high=10):
#     values = np.random.laplace(loc, scale, size)
#     #values = np.interp(values, (values.min(), values.max()), (low, high))
#     return values


# # def generate_induced_subgraph_game(distribution, n_agents, **kwargs):
# #     induced_subgraph_game = {}
# #     keys = list(itertools.combinations(range(1,n_agents+1), 2))
# #     totalinteractions = len(keys)
# #     values = distribution(totalinteractions, **kwargs)
# #     for i,key in enumerate(keys):
# #         induced_subgraph_game[','.join(map(str,key))] = round(values[i],2)
# #     return induced_subgraph_game

In [16]:
# def generate_induced_subgraph_game(distribution, n_agents, sparsity=0.0, seed = 123, **kwargs):
#     """
#     generate_induced_subgraph_game: To generate a . 
#     :distribution: distribution function name to sample the edge weights from. (dtype: function) 
#     :n_agents: number of agents (dtype: int) 
#     :sparsity: describes the connectivity of the graph edges. \
#     If 0.0, generates a fully connected graph. \
#     If 1.0 generates a connected graph with minimal edges. 
#     (dtype: float, range: [0,1]) 
#     :returns: induced subgraph game with agents pairs as key and their synergy as float value. 
#     (dtype: dictionary) 
#     """
#     induced_subgraph_game = {}
#     #### D: tyring to set a seed to get compareable results
#     connected_edges = [(i + 1, j + 1) for i, j in nx.random_tree(n_agents, seed=seed).edges] 
#     extra_edges = list(set(itertools.combinations(range(1, n_agents + 1), 2)).difference(connected_edges))
#     #### D: seed
#     rnd.seed(seed)
#     sampled_edges = rnd.sample(extra_edges, int((1 - sparsity) * len(extra_edges)))
#     keys = sorted(connected_edges + sampled_edges)
#     totalinteractions = len(keys)
#     np.random.seed(seed=seed)
#     values = distribution(totalinteractions, **kwargs)
#     for i, key in enumerate(keys):
#         induced_subgraph_game[','.join(map(str, key))] = round(values[i], 2)
#     G=nx.Graph()
#     G.add_nodes_from(np.arange(0,n_agents,1))
#     elist = [tuple((int(x)-1 for x in key.split(',')))+tuple([induced_subgraph_game[key]*-1]) for key in induced_subgraph_game]
#     G.add_weighted_edges_from(elist)
#     return induced_subgraph_game, G

In [17]:
import pickle

filename = 'starlink_tle_data.pkl'

def load_tle_data(filename):
    with open(filename, 'rb') as file:
        tle_data = pickle.load(file)
    return tle_data

# Example usage
# loaded_tle_data = load_tle_data(filename)
# loaded_tle_data

In [18]:
def fetch_tle_data():
    # url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
    url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle"
    # url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=iridium&FORMAT=tle"
    response = requests.get(url)
    lines = response.text.splitlines()
    tle_data = []
    for i in range(0, len(lines), 3):
        name = lines[i]
        line1 = lines[i + 1]
        line2 = lines[i + 2]
        tle_data.append((name, line1, line2))
    return tle_data

# Convert TLE data to satellite positions (latitude, longitude)
def tle_to_latlon(tle_data,n, timestamp):
    satellite_positions = []
    for name, line1, line2 in tle_data[:n]:  # Limit to first 10 satellites for example
        satellite = Satrec.twoline2rv(line1, line2)
        jd, fr = timestamp #jday(2024, 6, 25, 0, 0, 0)  # Example date
        e, r, v = satellite.sgp4(jd, fr)
        if e == 0:
            x, y, z = r
            lat = np.degrees(np.arcsin(z / np.linalg.norm(r)))
            lon = np.degrees(np.arctan2(y, x))
            satellite_positions.append((lat, lon))
            # print(f"Satellite {name}: lat {lat}, lon {lon}")  # Debugging print statement
        else:
            print(f"Error computing position for satellite {name}")
    return satellite_positions

# Convert latitude and longitude to Cartesian coordinates
def latlon_to_cartesian(lat, lon):
    lat = np.radians(lat)
    lon = np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.array([x, y, z])

# Calculate the great-circle distance between two points on a sphere
def great_circle_distance(point1, point2):
    return np.arccos(np.clip(np.dot(point1, point2), -1.0, 1.0))

# Construct the random geometric graph
def construct_graph(points, radius):
    G = nx.Graph()
    for i, point1 in enumerate(points):
        G.add_node(i, pos=point1)
        for j, point2 in enumerate(points[i+1:], i+1):
            dist = great_circle_distance(point1, point2)
            # if dist < radius:
            #     # Score calculation: Distance-based score modified by random interference factor
            #     score = np.exp(-dist) * (np.random.rand() * 2 - 1)  # Exponential decay with random factor
            #     G.add_weighted_edges_from([(i, j, score)])
            dist = great_circle_distance(point1, point2)
            if dist < radius:
                G.add_weighted_edges_from([(i, j, -dist)])
    return G

# Generate great-circle arc points between two points on the sphere
def generate_great_circle_arc(point1, point2, num_points=100):
    t = np.linspace(0, 1, num_points)
    omega = great_circle_distance(point1, point2)
    sin_omega = np.sin(omega)
    arc_points = (np.sin((1 - t) * omega)[:, None] * point1[None, :] +
                  np.sin(t * omega)[:, None] * point2[None, :]) / sin_omega
    return arc_points

# Plot the sphere, nodes, and edges
def plot_sphere_with_graph(points, graph):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the sphere
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
    x = np.cos(u) * np.sin(v)
    y = np.sin(u) * np.sin(v)
    z = np.cos(v)
    ax.plot_surface(x, y, z, color='c', alpha=0.3)
    
    # Plot the nodes
    for point in points:
        ax.scatter(point[0], point[1], point[2], color='b', s=50)
    
    # Plot the edges
    for edge in graph.edges():
        point1 = points[edge[0]]
        point2 = points[edge[1]]
        arc_points = generate_great_circle_arc(point1, point2)
        ax.plot(arc_points[:, 0], arc_points[:, 1], arc_points[:, 2], color='r')
    
    plt.show()


def get_graph_positions(timestamp, n, radius):
    # tle_data = fetch_tle_data()
    filename = 'starlink_tle_data.pkl'
    tle_data = load_tle_data(filename)
    satellite_positions = tle_to_latlon(tle_data, n, timestamp)
    points = np.array([latlon_to_cartesian(lat, lon) for lat, lon in satellite_positions])
    graph = construct_graph(points, radius)
    return graph, points

# # Main function
# tle_data = fetch_tle_data()
# n = 15
# timestamp = jday(2024, 6, 25, 0, 0, 0)
# satellite_positions = tle_to_latlon(tle_data, n, timestamp)
# print(len(satellite_positions))

# # Check if we got valid positions
# if not satellite_positions:
#     print("No satellite positions found. Check TLE data and sgp4 library setup.")
# else:
#     points = np.array([latlon_to_cartesian(lat, lon) for lat, lon in satellite_positions])

#     # Construct the graph
#     radius = 1.0  # Adjust this radius to change the connectivity
#     graph = construct_graph(points, radius)

#     # Plot the graph on the sphere
#     plot_sphere_with_graph(points, graph)

In [19]:
timestamp = jday(2024, 6, 1, 0, 0, 0)
n = 10
radius = 0.5

G, pos = get_graph_positions(timestamp, n, radius)

Error computing position for satellite STARLINK-1007           


In [20]:
# plot_sphere_with_graph(pos, G)

In [21]:
# connected_components = list(nx.connected_components(G))
# connected_components

In [22]:
# # Step 2: Create subgraphs for each connected component
# subgraphs = [G.subgraph(component).copy() for component in connected_components]

# # subgraphs now contains the list of disconnected subgraphs
# for i, subgraph in enumerate(subgraphs):
#     print(f"Subgraph {i+1}:")
#     print(subgraph.edges(data=True))

## Experiments

In [23]:
# #@title Choose the solvers for experiments
# IDP_brute_force = False#@param {type:"boolean"}
# IDP_topdown_min_cut = False#@param {type:"boolean"}
# IDP_topdown_qubo = False#@param {type:"boolean"}
# IDP_min_cut_dwave_annealer = True#@param {type:"boolean"}
# IDP_min_cut_gurobi = True#@param {type:"boolean"}


# solver_flags = ''.join(map(str,map(int,[IDP_brute_force,IDP_topdown_min_cut,IDP_topdown_qubo,IDP_min_cut_dwave_annealer,IDP_min_cut_gurobi])))

# #@markdown IDP implicitly means, it's using bottom-up approach.
# #@markdown IDP_min_cut_qiskit_qaoa takes the longest time for execution.

In [24]:
#report_save_location = '/content/drive/MyDrive/Saarland/QAI/DP-Q'

In [25]:
# now = datetime.now()

In [26]:
# seed = 111

# dwave_annealer_solver = True#@param {type:"boolean"}
# gurobipy_qubo_solver = True#@param {type:"boolean"}

# solver_flags = ''.join(map(str,map(int,[dwave_annealer_solver, gurobipy_qubo_solver])))


# table_contents = []

# base_path = datetime.now().strftime("%d%m%Y%H%M%S")

# # heights = np.arange(2, 51).tolist()
# report_filename = 'optimalSplit_report_' + solver_flags + '_' +  str(seed) + '.txt'
# report_filename_csv = os.path.join(base_path , 'FullReport_' + solver_flags + '_' +  str(seed) + '.csv')
# # qputimes_filename = 'optimalSplit_qputimes_' + solver_flags + '_' +  str(seed) + '.csv'

# qpu_output_path = os.path.join('singleSplitresults', f'{seed}')

# n_samples = 2000

In [27]:
# # Create the directory if it doesn't exist
# # os.makedirs('optimalSplitResults', exist_ok=True)

# fieldnames = ['n', 'sparsity', 'distribution', 'seed', 'solution_first', 'value_first', 'solution_lowest', 'value_lowest', 'annealer_tte', 'solution_gurobi', 'value_gurobi', 'gurobi_tte']
# if not os.path.exists(report_filename_csv):
#   with open(report_filename_csv, mode='w', newline='') as report_file:
#         writer = csv.DictWriter(report_file, fieldnames=fieldnames)
#         # Write the header row
#         writer.writeheader()

In [28]:
# # Create the directory if it doesn't exist
# os.makedirs(qpu_output_path, exist_ok=True)

# fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'n']
# if dwave_annealer_solver and not os.path.exists(qputimes_filename):
#   with open(qputimes_filename, mode='w', newline='') as qputime_file:
#         writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
#         # Write the header row
#         writer.writeheader()

In [29]:
# radius = 1.0
# timestamp = jday(2024, 7, 1, 0, 0, 0)
# n_satellites = np.arange(5, 51).tolist()

# n_max = 5

In [30]:
def get_coalition_value(coalition, induced_subgraph_game):
    # agents = coalition.split(',')
    return sum([induced_subgraph_game[key] for key in itertools.combinations(coalition, 2) if key in induced_subgraph_game])


In [31]:
"Done"

'Done'

In [32]:
n_satellites = np.arange(2, 51).tolist()
n = 40

radius = 1.0
timestamp = jday(2024, 7, 1, 0, 0, 0)

n_max = 5

G, pos = get_graph_positions(timestamp, n, radius)

split_counter = 0

base_path = datetime.now().strftime("%d%m%Y%H%M%S") + f"_{n}"

os.makedirs(base_path, exist_ok=True)

# fieldnames = ['n', 'sparsity', 'distribution', 'seed', 'solution_annealer', 'value_annealer', 'annealer_tte', 'solution_gurobi', 'value_gurobi', 'gurobi_tte']
# if not os.path.exists(os.path.join(base_path,report_filename_csv)):
#   with open(report_filename_csv, mode='w', newline='') as report_file:
#         writer = csv.DictWriter(report_file, fieldnames=fieldnames)
#         # Write the header row
#         writer.writeheader()

qputimes_filename = os.path.join(base_path, 'Split_qputimes.csv')

fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'split_index', 'n']
if not os.path.exists(qputimes_filename):
  with open(qputimes_filename, mode='w', newline='') as qputime_file:
        writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
        # Write the header row
        writer.writeheader()

In [33]:
isg = {}
for node1,node2,weight in G.edges(data=True):
    isg[node1,node2] = weight['weight']

In [ ]:
[[17, 11, 4, 13], [35], [20, 28, 21, 25, 38], [36, 29], [3, 34, 26, 5], [32, 9, 14, 31], [24, 33, 19, 12], [18, 6, 37], [39, 22, 15], [0, 16, 30, 7], [27, 1, 10], [8, 2, 23]]

In [32]:
start_time = time.time()
annealer_solution = get_optimal_coalition_structure_annealer(G, n_max)
dwave_annealer_tte = (time.time() - start_time)
print("annealer_solution",annealer_solution)
print("value",sum([get_coalition_value(coalition, isg) for coalition in annealer_solution]))
print('dwave_annealer_tte',dwave_annealer_tte)

part1, part2 [0, 24, 27, 33, 39, 1, 7, 8, 18, 19, 22, 2, 6, 10, 12, 16, 23, 30, 37, 15] [3, 4, 9, 20, 28, 29, 31, 34, 35, 26, 36, 17, 21, 5, 11, 25, 32, 13, 14, 38]
part1, part2 [3, 9, 29, 31, 34, 26, 36, 5, 32, 14] [4, 20, 28, 35, 17, 21, 11, 25, 13, 38]
part1, part2 [20, 28, 35, 21, 25, 38] [4, 17, 11, 13]
part1, part2 [20, 28, 21, 25, 38] [35]
part1, part2 [9, 31, 32, 14] [3, 29, 34, 26, 36, 5]
part1, part2 [3, 34, 26, 5] [29, 36]
part1, part2 [0, 27, 1, 7, 8, 2, 10, 16, 23, 30] [24, 33, 39, 18, 19, 22, 6, 12, 37, 15]
part1, part2 [39, 18, 22, 6, 37, 15] [24, 33, 19, 12]
part1, part2 [39, 22, 15] [18, 6, 37]
part1, part2 [27, 1, 8, 2, 10, 23] [0, 7, 16, 30]
part1, part2 [8, 2, 23] [27, 1, 10]
annealer_solution [[17, 11, 4, 13], [35], [20, 28, 21, 25, 38], [36, 29], [3, 34, 26, 5], [32, 9, 14, 31], [24, 33, 19, 12], [18, 6, 37], [39, 22, 15], [0, 16, 30, 7], [27, 1, 10], [8, 2, 23]]
value -0.2081737817550926
dwave_annealer_tte 58.55770564079285


In [ ]:
start_time = time.time()
gurobi_solution = get_optimal_coalition_structure_gurobi(G, n_max)
gurobi_tte = (time.time() - start_time)
print("gurobi_solution",gurobi_solution)
print("value",sum([get_coalition_value(coalition, isg) for coalition in gurobi_solution]))
print('gurobi_tte',gurobi_tte)

In [37]:
n_satellites = np.arange(2, 51).tolist()

In [38]:
def count_edges_in_subgraphs(graph, vertex_sets):
    subgraph_edge_count = 0
    for vertex_set in vertex_sets:
        subgraph = graph.subgraph(vertex_set)
        subgraph_edge_count += subgraph.number_of_edges()
    return subgraph_edge_count

In [40]:
for n in n_satellites:
  radius = 1.0
  timestamp = jday(2024, 7, 1, 0, 0, 0)

  n_max = 5

  G, pos = get_graph_positions(timestamp, n, radius)

  split_counter = 0

  base_path = datetime.now().strftime("%d%m%Y%H%M%S") + f"_{n}"

  os.makedirs(base_path, exist_ok=True)

  # fieldnames = ['n', 'sparsity', 'distribution', 'seed', 'solution_annealer', 'value_annealer', 'annealer_tte', 'solution_gurobi', 'value_gurobi', 'gurobi_tte']
  # if not os.path.exists(os.path.join(base_path,report_filename_csv)):
  #   with open(report_filename_csv, mode='w', newline='') as report_file:
  #         writer = csv.DictWriter(report_file, fieldnames=fieldnames)
  #         # Write the header row
  #         writer.writeheader()

  qputimes_filename = os.path.join(base_path, 'Split_qputimes.csv')

  fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'split_index', 'n']
  if not os.path.exists(qputimes_filename):
    with open(qputimes_filename, mode='w', newline='') as qputime_file:
          writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
          # Write the header row
          writer.writeheader()

  isg = {}
  for node1,node2,weight in G.edges(data=True):
      isg[node1,node2] = weight['weight']

  start_time = time.time()
  annealer_solution = get_optimal_coalition_structure_annealer(G, n_max)
  dwave_annealer_tte = (time.time() - start_time)
  # print("annealer_solution",annealer_solution)
  # print("value",sum([get_coalition_value(coalition, isg) for coalition in annealer_solution]))
  # print('dwave_annealer_tte',dwave_annealer_tte)

  print(f"\n\n\n\n n = {n}")
  print(f"Total edges = {G.number_of_edges()}")
  print(f"After Coalition = {count_edges_in_subgraphs(G,annealer_solution)}")





 n = 2
Total edges = 0
After Coalition = 0




 n = 3
Total edges = 0
After Coalition = 0




 n = 4
Total edges = 2
After Coalition = 2




 n = 5
Total edges = 4
After Coalition = 4




 n = 6
Total edges = 4
After Coalition = 4




 n = 7
Total edges = 6
After Coalition = 6




 n = 8
Total edges = 9
After Coalition = 9
part1, part2 [3, 4] [0, 1, 7, 8]




 n = 9
Total edges = 13
After Coalition = 6
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 10
Total edges = 17
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 11
Total edges = 18
After Coalition = 10
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 12
Total edges = 21
After Coalition = 13
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12] [2, 5, 11]




 n = 13
Total edges = 24
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12] [2, 5, 11, 13]




 n = 14
Total edges = 26
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12, 14] [2, 5, 11, 13]




 n 

In [31]:
n_satellites = np.arange(10, 201).tolist()
# n = 40
for n in n_satellites:
  # try:
    print(f"\n Exceuting for {n} satellites")
    radius = 1.0
    timestamp = jday(2024, 7, 1, 0, 0, 0)

    n_max = 5

    G, pos = get_graph_positions(timestamp, n, radius)

    split_counter = 0

    base_path = datetime.now().strftime("%d%m%Y%H%M%S") + f"_{n}"

    os.makedirs(base_path, exist_ok=True)

    # fieldnames = ['n', 'sparsity', 'distribution', 'seed', 'solution_annealer', 'value_annealer', 'annealer_tte', 'solution_gurobi', 'value_gurobi', 'gurobi_tte']
    # if not os.path.exists(os.path.join(base_path,report_filename_csv)):
    #   with open(report_filename_csv, mode='w', newline='') as report_file:
    #         writer = csv.DictWriter(report_file, fieldnames=fieldnames)
    #         # Write the header row
    #         writer.writeheader()

    qputimes_filename = os.path.join(base_path, 'Split_qputimes.csv')

    fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'split_index', 'n']
    if not os.path.exists(qputimes_filename):
      with open(qputimes_filename, mode='w', newline='') as qputime_file:
            writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
            # Write the header row
            writer.writeheader()

    isg = {}
    for node1,node2,weight in G.edges(data=True):
        isg[node1,node2] = weight['weight']

    start_time = time.time()
    annealer_solution = get_optimal_coalition_structure_annealer(G, n_max)
    dwave_annealer_tte = (time.time() - start_time)
    print("annealer_solution",annealer_solution)
    print("value",sum([get_coalition_value(coalition, isg) for coalition in annealer_solution]))
    print('dwave_annealer_tte',dwave_annealer_tte)
  # except:
    # continue


 Exceuting for 10 satellites


IndexError: list index out of range

In [3]:
for n in n_satellites:
    # print(f'n. agents: {n}',end='\n')
    # np.random.seed(seed=seed)
    # induced_subgraph_game = utils.generate_induced_subgraph_game(distribution,n)
    # problem_instances[distribution][n] = induced_subgraph_game
    optimal_coalition_structure = []
    
    G, pos = get_graph_positions(timestamp, n, radius)
    connected_components = list(nx.connected_components(G))

    # induced_subgraph_games = []
    # for component in connected_components:
        # if len(component) > n_max:
            # isg = {}
            # # G.subgraph(component).edges(data=True)
            # for node1,node2,weight in G.subgraph(component).edges(data=True):
                # isg[','.join(map(str,[node1,node2]))] = weight['weight']
            # induced_subgraph_games.append(isg)
        # else:
            # optimal_coalition_structure.append(component)

    # 
    # for induced_subgraph_game in induced_subgraph_games:
        
    


    # start_time = time.time()
    # if IDP_brute_force:
    #     bruteforce_solution, bruteforce_value = IDP(coalition_game)
    # else:
    #     bruteforce_solution, bruteforce_value = None, None
    # bruteforce_tte = (time.time() - start_time)

    # print('--- total time bruteforce:', time.strftime('%H:%M:%S', time.gmtime(bruteforce_tte)), '\n')

    # start_time = time.time()
    # if IDP_topdown_min_cut:
        # topdown_min_cut_solution, topdown_min_cut_value = IDP_min_cut_top_down(induced_subgraph_game)
        # if bruteforce_value:
            # try:
                # topdown_min_cut_quality = 1 -(abs(topdown_min_cut_value-bruteforce_value)/bruteforce_value)
            # except:
                # topdown_min_cut_quality = 1-(abs(topdown_min_cut_value-bruteforce_value))
        # else:
            # topdown_min_cut_quality = 1-(abs(topdown_min_cut_value-bruteforce_value))
    # else:
        # topdown_min_cut_solution, topdown_min_cut_value = None, None
        # topdown_min_cut_quality = None
    # topdown_min_cut_tte = (time.time() - start_time)
    #print('--- BF min-cut top-down:', time.strftime('%H:%M:%S', time.gmtime(topdown_min_cut_tte)), '\n')

    # start_time = time.time()
    # if IDP_topdown_qubo:
        # topdown_qubo_solution, topdown_qubo_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                        #  min_cut_solver = min_cut_solvers.min_cut_qiskit_classical_eigensolver)
        # if bruteforce_value:
            # try:
                # topdown_qubo_quality = 1-(abs(topdown_qubo_value-bruteforce_value)/bruteforce_value)
            # except:
                # topdown_qubo_quality =1-(abs(topdown_qubo_value-bruteforce_value))
        # else:
            # topdown_qubo_quality = 1-(abs(topdown_qubo_value-bruteforce_value))
    # else:
        # topdown_qubo_solution, topdown_qubo_value = None, None
    # topdown_qubo_tte = (time.time() - start_time)
    #print('--- total min-cut qubo:', time.strftime('%H:%M:%S', time.gmtime(topdown_qubo_tte)), '\n')

    start_time = time.time()
    #Call DWAVE ANNEALER SOLVER
    #dwave_annealer_solution, dwave_annealer_value = np.array([0]*n),None
    if IDP_min_cut_gurobi:
        split_counter = 0
        gurobi_solution, gurobi_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                             min_cut_solver = min_cut_solvers.gurobi_solver)
        gurobi_quality = 1
        # # dwave_annealer_quality = None
        # if IDP_topdown_min_cut:
        #     try:
        #         gurobi_quality = 1-(abs(gurobi_value-bruteforce_value)/bruteforce_value)
        #     except:
        #         gurobi_quality =1-(abs(gurobi_value-bruteforce_value))
        # else:
        #     gurobi_quality = 1-(abs(gurobi_value-bruteforce_value))
    else:
        gurobi_solution, gurobi_value = None, None
        # gurobi_quality = ((gurobi_solution==gurobi_solution).all() or (gurobi_solution==(1-gurobi_solution)).all())
        gurobi_quality = 1
    gurobi_tte = (time.time() - start_time)

    start_time = time.time()
    #Call DWAVE ANNEALER SOLVER
    #dwave_annealer_solution, dwave_annealer_value = np.array([0]*n),None
    if IDP_min_cut_dwave_annealer:
        dwave_annealer_solution, dwave_annealer_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                             min_cut_solver = min_cut_solvers.min_cut_dwave_annealer)
        # dwave_annealer_quality = None
        if IDP_topdown_min_cut:
            try:
                dwave_annealer_quality = 1-(abs(dwave_annealer_value-gurobi_value)/gurobi_value)
            except:
                dwave_annealer_quality =1-(abs(dwave_annealer_value-gurobi_value))
        else:
            dwave_annealer_quality = 1-(abs(dwave_annealer_value-gurobi_value))
    else:
        dwave_annealer_solution, dwave_annealer_value = None, None
        dwave_annealer_quality = ((bruteforce_solution==dwave_annealer_solution).all() or (bruteforce_solution==(1-dwave_annealer_solution)).all())
    dwave_annealer_tte = (time.time() - start_time)

    print(f'n. agents: {n}', '--- total time QA:', time.strftime('%H:%M:%S', time.gmtime(dwave_annealer_tte)), end="\n")

    row = []
    row.append(distribution.__name__)
    row.append(n)
    if IDP_brute_force:
        row.append(str(bruteforce_solution))
        row.append(bruteforce_value)
        row.append(bruteforce_tte)
    if IDP_topdown_min_cut:
        row.append(str(topdown_min_cut_solution))
        row.append(topdown_min_cut_value)
        row.append(topdown_min_cut_tte)
        row.append(topdown_min_cut_quality)
    if IDP_topdown_qubo:
        row.append(str(topdown_qubo_solution))
        row.append(topdown_qubo_value)
        row.append(topdown_qubo_tte)
        row.append(topdown_qubo_quality)
    if IDP_min_cut_dwave_annealer:
        row.append(str(dwave_annealer_solution))
        row.append(dwave_annealer_value)
        row.append(dwave_annealer_tte)
        row.append(dwave_annealer_quality)
    #report_file_obj = open(os.path.join(report_save_location,report_filename),'a+')
    report_file_obj = open(os.path.join(report_filename),'a+')
    report_file_obj.write('__'.join(map(str,row))+'\n')
    report_file_obj.close()
    table_contents.append(row)
print('\n')

NameError: name 'n_agents' is not defined

In [ ]:
table_contents = []


distributions = [  
    laplace,
    normal
]

 
n_agents = np.arange(2, 41).tolist()

seed = 123

report_filename = 'IDP_report_' + solver_flags + '_' +  str(seed) + '.txt'

problem_instances = {}

# if generate_file_for_BOSS:
    # file_obj = open(f"data_for_BOSS_{seed}.txt",'w')


for iteration in range(1,4):
    try:
        print('*****  iteration  *****', iteration)
        for distribution in distributions:
            print(f'\nExecuting {distribution.__name__} distribution ',end=' ')
            problem_instances[distribution] = {}

            for n in n_agents:
                # print(f'n. agents: {n}',end='\n')
                np.random.seed(seed=seed)
                induced_subgraph_game = utils.generate_induced_subgraph_game(distribution,n)

                problem_instances[distribution][n] = induced_subgraph_game

                start_time = time.time()
                if IDP_brute_force:
                    bruteforce_solution, bruteforce_value = IDP(coalition_game)
                else:
                    bruteforce_solution, bruteforce_value = None, None
                bruteforce_tte = (time.time() - start_time)
                #print('--- total time bruteforce:', time.strftime('%H:%M:%S', time.gmtime(bruteforce_tte)), '\n')

                start_time = time.time()
                if IDP_topdown_min_cut:
                    topdown_min_cut_solution, topdown_min_cut_value = IDP_min_cut_top_down(induced_subgraph_game)
                    if bruteforce_value:
                        try:
                            topdown_min_cut_quality = 1 -(abs(topdown_min_cut_value-bruteforce_value)/bruteforce_value)
                        except:
                            topdown_min_cut_quality = 1-(abs(topdown_min_cut_value-bruteforce_value))
                    else:
                        topdown_min_cut_quality = 1-(abs(topdown_min_cut_value-bruteforce_value))
                else:
                    topdown_min_cut_solution, topdown_min_cut_value = None, None
                    topdown_min_cut_quality = None
                topdown_min_cut_tte = (time.time() - start_time)
                #print('--- BF min-cut top-down:', time.strftime('%H:%M:%S', time.gmtime(topdown_min_cut_tte)), '\n')

                start_time = time.time()
                if IDP_topdown_qubo:
                    topdown_qubo_solution, topdown_qubo_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                                     min_cut_solver = min_cut_solvers.min_cut_qiskit_classical_eigensolver)
                    if bruteforce_value:
                        try:
                            topdown_qubo_quality = 1-(abs(topdown_qubo_value-bruteforce_value)/bruteforce_value)
                        except:
                            topdown_qubo_quality =1-(abs(topdown_qubo_value-bruteforce_value))
                    else:
                        topdown_qubo_quality = 1-(abs(topdown_qubo_value-bruteforce_value))
                else:
                    topdown_qubo_solution, topdown_qubo_value = None, None
                topdown_qubo_tte = (time.time() - start_time)
                #print('--- total min-cut qubo:', time.strftime('%H:%M:%S', time.gmtime(topdown_qubo_tte)), '\n')


                start_time = time.time()
                #Call DWAVE ANNEALER SOLVER
                #dwave_annealer_solution, dwave_annealer_value = np.array([0]*n),None
                if IDP_min_cut_gurobi:
                    split_counter = 0
                    gurobi_solution, gurobi_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                                         min_cut_solver = min_cut_solvers.gurobi_solver)
                    gurobi_quality = 1
                    # # dwave_annealer_quality = None
                    # if IDP_topdown_min_cut:
                    #     try:
                    #         gurobi_quality = 1-(abs(gurobi_value-bruteforce_value)/bruteforce_value)
                    #     except:
                    #         gurobi_quality =1-(abs(gurobi_value-bruteforce_value))
                    # else:
                    #     gurobi_quality = 1-(abs(gurobi_value-bruteforce_value))
                else:
                    gurobi_solution, gurobi_value = None, None
                    # gurobi_quality = ((gurobi_solution==gurobi_solution).all() or (gurobi_solution==(1-gurobi_solution)).all())
                    gurobi_quality = 1
                gurobi_tte = (time.time() - start_time)


                start_time = time.time()
                #Call DWAVE ANNEALER SOLVER
                #dwave_annealer_solution, dwave_annealer_value = np.array([0]*n),None
                if IDP_min_cut_dwave_annealer:
                    dwave_annealer_solution, dwave_annealer_value = IDP_min_cut_top_down(induced_subgraph_game, 
                                                                                         min_cut_solver = min_cut_solvers.min_cut_dwave_annealer)
                    # dwave_annealer_quality = None
                    if IDP_topdown_min_cut:
                        try:
                            dwave_annealer_quality = 1-(abs(dwave_annealer_value-gurobi_value)/gurobi_value)
                        except:
                            dwave_annealer_quality =1-(abs(dwave_annealer_value-gurobi_value))
                    else:
                        dwave_annealer_quality = 1-(abs(dwave_annealer_value-gurobi_value))
                else:
                    dwave_annealer_solution, dwave_annealer_value = None, None
                    dwave_annealer_quality = ((bruteforce_solution==dwave_annealer_solution).all() or (bruteforce_solution==(1-dwave_annealer_solution)).all())
                dwave_annealer_tte = (time.time() - start_time)


                print(f'n. agents: {n}', '--- total time QA:', time.strftime('%H:%M:%S', time.gmtime(dwave_annealer_tte)), end="\n")

                row = []
                row.append(distribution.__name__)
                row.append(n)
                if IDP_brute_force:
                    row.append(str(bruteforce_solution))
                    row.append(bruteforce_value)
                    row.append(bruteforce_tte)

                if IDP_topdown_min_cut:
                    row.append(str(topdown_min_cut_solution))
                    row.append(topdown_min_cut_value)
                    row.append(topdown_min_cut_tte)
                    row.append(topdown_min_cut_quality)

                if IDP_topdown_qubo:
                    row.append(str(topdown_qubo_solution))
                    row.append(topdown_qubo_value)
                    row.append(topdown_qubo_tte)
                    row.append(topdown_qubo_quality)

                if IDP_min_cut_dwave_annealer:
                    row.append(str(dwave_annealer_solution))
                    row.append(dwave_annealer_value)
                    row.append(dwave_annealer_tte)
                    row.append(dwave_annealer_quality)
                #report_file_obj = open(os.path.join(report_save_location,report_filename),'a+')
                report_file_obj = open(os.path.join(report_filename),'a+')
                report_file_obj.write('__'.join(map(str,row))+'\n')
                report_file_obj.close()
                table_contents.append(row)
            print('\n')
        if generate_file_for_BOSS:
            file_obj.close()
    except:
        print("iteration ", iteration, "not working"  )

## Display results from generated report file

In [18]:
report_filename = 'IDP_report_' + solver_flags + '_' +  str(seed) + '.txt'
report_filename

'IDP_report_0001_123.txt'

In [19]:
IDP_brute_force = bool(int(report_filename.split('_')[-2][0]))
IDP_topdown_min_cut = bool(int(report_filename.split('_')[-2][1]))
IDP_topdown_qubo = bool(int(report_filename.split('_')[-2][2]))
IDP_min_cut_dwave_annealer = bool(int(report_filename.split('_')[-2][3]))

#report_file_obj = open(os.path.join(report_save_location,report_filename),'r')
report_file_obj = open(os.path.join(report_filename),'r')
table_contents = [line.replace('\n','').split('__') for line in report_file_obj.readlines()]

In [20]:
def is_float(value):
  try:
    float(value)
    return True
  except:
    return False

for col_num, cell in enumerate(table_contents[0][2:]):
  if is_float(cell):
    for row in table_contents:
      row[col_num+2] = np.float(row[col_num+2])

In [ ]:
#view output table
def highlight_false(s, column):
    is_false = pd.Series(data=False, index=s.index)
    is_false[column] = round(s.loc[column],2)<1.0
    return ['color: #ff8888' if is_false.any() else '' for v in is_false]

base_cols = ['Distribution', 'No. of Agents']
sub_cols = ['', '']

if IDP_brute_force:
    base_cols = base_cols+['Brute Force']*3
    sub_cols=sub_cols+['Result', 'Value', 'TTE']
if IDP_topdown_min_cut:
    base_cols = base_cols+['Top-down approach using min-cut']*4
    sub_cols=sub_cols+['Result', 'Value', 'TTE', 'Quality']
if IDP_topdown_qubo:
    base_cols = base_cols+['Top-down approach using qubo']*4
    sub_cols=sub_cols+['Result', 'Value', 'TTE', 'Quality']
if IDP_min_cut_dwave_annealer:
    base_cols = base_cols+['Dwave Annealer']*4
    sub_cols=sub_cols+['Result', 'Value', 'TTE', 'Quality']


column_arrays = [base_cols, sub_cols]


#df = pd.DataFrame(table_contents, columns=table_headers)
df = pd.DataFrame(table_contents, columns=pd.MultiIndex.from_arrays(column_arrays))


s = df.style.apply(highlight_false, column=('Top-down approach using min-cut', 'Quality'), axis=1)

cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', 'grey')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #1D1D1D; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

result_col_bgcolor = '186A3B'
value_col_bgcolor = '784212'
tte_col_bgcolor = '154360'
correctness_col_bgcolor = '693f3f'

def get_nested_column_style(col_name, nested_col_name, border_color='black', bg_color = 'grey'):
    return {(col_name, nested_col_name):[{'selector': 'th', 'props': 'border-left: 1px solid '+border_color},
                                         {'selector': 'td', 'props': 'border-left: 1px solid '+ border_color+'; background-color: #'+bg_color}]}

def get_column_style(col_name, nested_col_names):
    result_dict = {}
    for nested_col_name, bg_color in nested_col_names:
        border_color = 'black'
        if nested_col_name is 'Result':
            border_color = 'white'
        temp = get_nested_column_style(col_name, nested_col_name, border_color, bg_color)
        result_dict[list(temp.keys())[0]]=list(temp.values())[0]
    return result_dict

d1 = {
    ('No. of Agents', ''): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid white'}]}

if IDP_brute_force: d1.update(get_column_style('Brute Force',[('Result',result_col_bgcolor), ('Value',value_col_bgcolor), ('TTE',tte_col_bgcolor)]))
if IDP_topdown_min_cut: d1.update(get_column_style('Top-down approach using min-cut',[('Result',result_col_bgcolor), ('Value',value_col_bgcolor), ('TTE',tte_col_bgcolor), ('Correctness',correctness_col_bgcolor)]))
if IDP_topdown_qubo: d1.update(get_column_style('Top-down approach using qubo',[('Result',result_col_bgcolor), ('Value',value_col_bgcolor), ('TTE',tte_col_bgcolor), ('Correctness',correctness_col_bgcolor)]))
if IDP_min_cut_dwave_annealer: d1.update(get_column_style('Qiskit QAOA',[('Result',result_col_bgcolor), ('Value',value_col_bgcolor), ('TTE',tte_col_bgcolor), ('Correctness',correctness_col_bgcolor), ('reps (p)','1D1D1D')]))

s.set_table_styles(d1, overwrite=False, axis=0)

In [ ]:
import matplotlib.pyplot as plt
for distr in df.Distribution.unique():
    data = df[df.Distribution == distr]
    
    
    plt.figure(figsize=(14,6))
    x = data['No. of Agents'].to_numpy()
    y_BF = data[('Top-down approach using min-cut', 'TTE')].astype(float).to_numpy()
    y_QA = data[('Dwave Annealer', 'TTE')].astype(float).to_numpy()
    
    plt.scatter(x, y_BF, label='Brute Force')
    plt.plot(x, y_BF)
    
    plt.scatter(x, y_QA, label='QA')
    plt.plot(x, y_QA)
    
    plt.grid()
    plt.title(distr)
    plt.legend()
    # plt.savefig(distr+'.png')
    plt.show()
    
    plt.figure(figsize=(14,6))

    log_BF = np.log(y_BF+1)
    log_QA = np.log(y_QA+1)
    
    plt.scatter(x, log_BF, label='Brute Force')
    plt.plot(x, log_BF)

    plt.scatter(x, log_QA, label='Quantum')
    plt.plot(x, log_QA)
    
    plt.grid()
    plt.title(distr + ' (log)')
    plt.legend()
    #plt.savefig(distr+'_log.png')
    plt.show()